In [ ]:
# !pip install transformers
# !pip install datasets

In [84]:
import pandas as pd
import torch
import datasets
import transformers
from transformers import AutoTokenizer,AutoModelForMaskedLM
from transformers import DataCollatorForLanguageModeling
from transformers import TrainingArguments
from transformers import Trainer
import warnings
warnings.filterwarnings("ignore")

In [85]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


In [86]:
train_df=pd.read_csv('EmbibleDataset/train_df.csv', sep='\\t', encoding='utf-8')
valid_df=pd.read_csv('EmbibleDataset/valid_df.csv', sep='\\t', encoding='utf-8')

In [87]:
train_df=train_df.drop(columns=["verse_idx.1","verse.1","name"])
#train_df["name.1"].rename("name")
train_df.rename(columns={"name.1": 'name'}, inplace=True)
#valid_df=train_df.drop(columns=["verse_idx.1","verse.1","name"])
#train_df["name.1"].rename("name")
#valid_df.rename(columns={"name.1": 'name'}, inplace=True)

In [88]:
print(train_df)
print(valid_df)

      verse_idx   verse                                               name
0        Isaiah     0.0  חֲזוֹן֙ יְשַֽׁעְיָ֣הוּ בֶן אָמ֔וֹץ אֲשֶׁ֣ר חָז...
1        Isaiah     1.0  שִׁמְע֤וּ שָׁמַ֙יִם֙ וְהַאֲזִ֣ינִי אֶ֔רֶץ כִּ֥...
2        Isaiah     2.0  יָדַ֥ע שׁוֹר֙ קֹנֵ֔הוּ וַחֲמ֖וֹר אֵב֣וּס בְּעָ...
3        Isaiah     3.0  ה֣וֹי גּ֣וֹי חֹטֵ֗א עַ֚ם כֶּ֣בֶד עָוֺ֔ן זֶ֣רַע...
4        Isaiah     4.0  עַ֣ל מֶ֥ה תֻכּ֛וּ ע֖וֹד תּוֹסִ֣יפוּ סָרָ֑ה כָּ...
...         ...     ...                                                ...
22145    Exodus  1208.0  וַיְכַ֥ס הֶעָנָ֖ן אֶת אֹ֣הֶל מוֹעֵ֑ד וּכְב֣וֹד...
22146    Exodus  1209.0  וְלֹא יָכֹ֣ל מֹשֶׁ֗ה לָבוֹא֙ אֶל אֹ֣הֶל מוֹעֵ֔...
22147    Exodus  1210.0  וּבְהֵעָל֤וֹת הֶֽעָנָן֙ מֵעַ֣ל הַמִּשְׁכָּ֔ן י...
22148    Exodus  1211.0  וְאִם לֹ֥א יֵעָלֶ֖ה הֶעָנָ֑ן וְלֹ֣א יִסְע֔וּ ע...
22149    Exodus  1212.0  כִּי֩ עֲנַ֨ן יְהוָ֤ה עַֽל הַמִּשְׁכָּן֙ יוֹמָ֔...

[22150 rows x 3 columns]
         name  verse_idx                                              vers

In [43]:
cp = "tau/tavbert-he"
model=AutoModelForMaskedLM.from_pretrained(cp)
tokenizer=AutoTokenizer.from_pretrained(cp)

loading configuration file config.json from cache at /home/fono/.cache/huggingface/hub/models--tau--tavbert-he/snapshots/41265b09a862144b2517afdfd46da4388f1380df/config.json
Model config RobertaConfig {
  "_name_or_path": "tau/tavbert-he",
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 2050,
  "model_type": "roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "transformers_version": "4.25.1",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 345
}

loading weights file pytorch_model.bin from cache at /home/fono/.cache/huggingface/hub/models--tau--tavbert-he/snapsho

In [44]:
train_ds=datasets.Dataset.from_pandas(train_df)
valid_ds=datasets.Dataset.from_pandas(valid_df)

In [89]:

def tokenize_function(dataset):
  return tokenizer(str(dataset["verse"]))
tokenized_train_ds=train_ds.map(tokenize_function)
tokenized_valid_ds=valid_ds.map(tokenize_function)


  0%|          | 0/22150 [00:00<?, ?ex/s]

  0%|          | 0/536 [00:00<?, ?ex/s]

In [90]:
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm_probability=0.15)

In [50]:
batch_size = 64
# Show the training loss with every epoch
logging_steps = len(tokenized_train_ds)

training_args = TrainingArguments(
    output_dir="EmbibleModels/Embibert-finetuned-tavbert-10epochs",
    overwrite_output_dir=True,
    evaluation_strategy="epoch",
    logging_strategy="epoch",
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    push_to_hub=False,
    logging_steps=logging_steps,
    num_train_epochs=10

)


PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [51]:
# from sklearn.metrics import f1_score

# def compute_metrics(pred):
#   return {'f1:':pred}

In [52]:
import gc

gc.collect()

torch.cuda.empty_cache()
trainer = Trainer(
    model,
    training_args,
    train_dataset=tokenized_train_ds,
    eval_dataset=tokenized_valid_ds,
    data_collator=data_collator,
    tokenizer=tokenizer,
)

In [53]:
trainer.train()

The following columns in the training set don't have a corresponding argument in `RobertaForMaskedLM.forward` and have been ignored: verse_idx, verse, name. If verse_idx, verse, name are not expected by `RobertaForMaskedLM.forward`,  you can safely ignore this message.
***** Running training *****
  Num examples = 22150
  Num Epochs = 10
  Instantaneous batch size per device = 64
  Total train batch size (w. parallel, distributed & accumulation) = 64
  Gradient Accumulation steps = 1
  Total optimization steps = 3470
  Number of trainable parameters = 87489369
You're using a PreTrainedTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss
1,1.334800,6.708065
2,1.198900,9.746973
3,1.183300,12.123498
4,1.185900,11.782933
5,1.172900,12.033704
6,1.166000,12.216064
7,1.164700,13.117076
8,1.158200,12.655667
9,1.176300,12.528611
10,1.168000,12.847200


The following columns in the evaluation set don't have a corresponding argument in `RobertaForMaskedLM.forward` and have been ignored: verse_idx, name, verse, __index_level_0__. If verse_idx, name, verse, __index_level_0__ are not expected by `RobertaForMaskedLM.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 536
  Batch size = 64
Saving model checkpoint to EmbibleModels/Embibert-finetuned-tavbert-10epochs/checkpoint-500
Configuration saved in EmbibleModels/Embibert-finetuned-tavbert-10epochs/checkpoint-500/config.json
Model weights saved in EmbibleModels/Embibert-finetuned-tavbert-10epochs/checkpoint-500/pytorch_model.bin
tokenizer config file saved in EmbibleModels/Embibert-finetuned-tavbert-10epochs/checkpoint-500/tokenizer_config.json
Special tokens file saved in EmbibleModels/Embibert-finetuned-tavbert-10epochs/checkpoint-500/special_tokens_map.json
The following columns in the evaluation set don't have a corresponding argument in `Ro

TrainOutput(global_step=3470, training_loss=1.190897837702067, metrics={'train_runtime': 576.2079, 'train_samples_per_second': 384.41, 'train_steps_per_second': 6.022, 'total_flos': 910566259197228.0, 'train_loss': 1.190897837702067, 'epoch': 10.0})

In [54]:
trainer.save_model("EmbibleModels/Embibert-finetuned-tavbert-10epochs")
model.save_pretrained("EmbibleModels/Embibert-finetuned-tavbert-10epochs")

Saving model checkpoint to EmbibleModels/Embibert-finetuned-tavbert-10epochs
Configuration saved in EmbibleModels/Embibert-finetuned-tavbert-10epochs/config.json
Model weights saved in EmbibleModels/Embibert-finetuned-tavbert-10epochs/pytorch_model.bin
tokenizer config file saved in EmbibleModels/Embibert-finetuned-tavbert-10epochs/tokenizer_config.json
Special tokens file saved in EmbibleModels/Embibert-finetuned-tavbert-10epochs/special_tokens_map.json
Configuration saved in EmbibleModels/Embibert-finetuned-tavbert-10epochs/config.json
Model weights saved in EmbibleModels/Embibert-finetuned-tavbert-10epochs/pytorch_model.bin


In [74]:
cp = "tau/tavbert-he"
model=AutoModelForMaskedLM.from_pretrained(cp)
tokenizer=AutoTokenizer.from_pretrained(cp)

loading configuration file config.json from cache at /home/fono/.cache/huggingface/hub/models--tau--tavbert-he/snapshots/41265b09a862144b2517afdfd46da4388f1380df/config.json
Model config RobertaConfig {
  "_name_or_path": "tau/tavbert-he",
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 2050,
  "model_type": "roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "transformers_version": "4.25.1",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 345
}

loading weights file pytorch_model.bin from cache at /home/fono/.cache/huggingface/hub/models--tau--tavbert-he/snapsho

In [75]:
batch_size = 64
# Show the training loss with every epoch
logging_steps = len(tokenized_train_ds)

training_args = TrainingArguments(
    output_dir="EmbibleModels/Embibert-finetuned-tavbert-20epochs",
    overwrite_output_dir=True,
    evaluation_strategy="epoch",
    logging_strategy="epoch",
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    push_to_hub=False,
    logging_steps=logging_steps,
    num_train_epochs=20

)


PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [76]:
import gc

gc.collect()

torch.cuda.empty_cache()

In [78]:
import gc

gc.collect()

torch.cuda.empty_cache()
trainer = Trainer(
    model,
    training_args,
    train_dataset=tokenized_train_ds,
    eval_dataset=tokenized_valid_ds,
    data_collator=data_collator,
    tokenizer=tokenizer,
)
trainer.train()

The following columns in the training set don't have a corresponding argument in `RobertaForMaskedLM.forward` and have been ignored: verse_idx, verse, name. If verse_idx, verse, name are not expected by `RobertaForMaskedLM.forward`,  you can safely ignore this message.
***** Running training *****
  Num examples = 22150
  Num Epochs = 20
  Instantaneous batch size per device = 64
  Total train batch size (w. parallel, distributed & accumulation) = 64
  Gradient Accumulation steps = 1
  Total optimization steps = 6940
  Number of trainable parameters = 87489369


Epoch,Training Loss,Validation Loss
1,1.318700,5.967810
2,1.198700,9.797073
3,1.182800,12.348807
4,1.184700,12.969960
5,1.174300,14.016263
6,1.165200,13.316539
7,1.167000,14.486106
8,1.167800,14.302264
9,1.179300,14.592799
10,1.175300,14.134422


The following columns in the evaluation set don't have a corresponding argument in `RobertaForMaskedLM.forward` and have been ignored: verse_idx, name, verse, __index_level_0__. If verse_idx, name, verse, __index_level_0__ are not expected by `RobertaForMaskedLM.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 536
  Batch size = 64
Saving model checkpoint to EmbibleModels/Embibert-finetuned-tavbert-20epochs/checkpoint-500
Configuration saved in EmbibleModels/Embibert-finetuned-tavbert-20epochs/checkpoint-500/config.json
Model weights saved in EmbibleModels/Embibert-finetuned-tavbert-20epochs/checkpoint-500/pytorch_model.bin
tokenizer config file saved in EmbibleModels/Embibert-finetuned-tavbert-20epochs/checkpoint-500/tokenizer_config.json
Special tokens file saved in EmbibleModels/Embibert-finetuned-tavbert-20epochs/checkpoint-500/special_tokens_map.json
The following columns in the evaluation set don't have a corresponding argument in `Ro

TrainOutput(global_step=6940, training_loss=1.1795711550314076, metrics={'train_runtime': 996.3567, 'train_samples_per_second': 444.62, 'train_steps_per_second': 6.965, 'total_flos': 1821174657443172.0, 'train_loss': 1.1795711550314076, 'epoch': 20.0})

In [79]:
trainer.save_model("EmbibleModels/Embibert-finetuned-tavbert-20epochs")
model.save_pretrained("EmbibleModels/Embibert-finetuned-tavbert-20epochs")

Saving model checkpoint to EmbibleModels/Embibert-finetuned-tavbert-20epochs
Configuration saved in EmbibleModels/Embibert-finetuned-tavbert-20epochs/config.json
Model weights saved in EmbibleModels/Embibert-finetuned-tavbert-20epochs/pytorch_model.bin
tokenizer config file saved in EmbibleModels/Embibert-finetuned-tavbert-20epochs/tokenizer_config.json
Special tokens file saved in EmbibleModels/Embibert-finetuned-tavbert-20epochs/special_tokens_map.json
Configuration saved in EmbibleModels/Embibert-finetuned-tavbert-20epochs/config.json
Model weights saved in EmbibleModels/Embibert-finetuned-tavbert-20epochs/pytorch_model.bin


In [91]:
cp = "tau/tavbert-he"
model=AutoModelForMaskedLM.from_pretrained(cp)
tokenizer=AutoTokenizer.from_pretrained(cp)

loading configuration file config.json from cache at /home/fono/.cache/huggingface/hub/models--tau--tavbert-he/snapshots/41265b09a862144b2517afdfd46da4388f1380df/config.json
Model config RobertaConfig {
  "_name_or_path": "tau/tavbert-he",
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 2050,
  "model_type": "roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "transformers_version": "4.25.1",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 345
}

loading weights file pytorch_model.bin from cache at /home/fono/.cache/huggingface/hub/models--tau--tavbert-he/snapsho

In [92]:
batch_size = 64
# Show the training loss with every epoch
logging_steps = len(tokenized_train_ds)

training_args = TrainingArguments(
    output_dir="EmbibleModels/Embibert-finetuned-tavbert-50epochs",
    overwrite_output_dir=True,
    evaluation_strategy="epoch",
    logging_strategy="epoch",
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    push_to_hub=False,
    logging_steps=logging_steps,
    num_train_epochs=50
)


PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [94]:
import gc

gc.collect()

torch.cuda.empty_cache()

In [95]:
import gc

gc.collect()

torch.cuda.empty_cache()
trainer = Trainer(
    model,
    training_args,
    train_dataset=tokenized_train_ds,
    eval_dataset=tokenized_valid_ds,
    data_collator=data_collator,
    tokenizer=tokenizer,
)
trainer.train()

The following columns in the training set don't have a corresponding argument in `RobertaForMaskedLM.forward` and have been ignored: verse_idx, verse, name. If verse_idx, verse, name are not expected by `RobertaForMaskedLM.forward`,  you can safely ignore this message.
***** Running training *****
  Num examples = 22150
  Num Epochs = 50
  Instantaneous batch size per device = 64
  Total train batch size (w. parallel, distributed & accumulation) = 64
  Gradient Accumulation steps = 1
  Total optimization steps = 17350
  Number of trainable parameters = 87489369


Epoch,Training Loss,Validation Loss
1,1.390800,6.705500
2,1.197300,9.663179
3,1.184900,11.419830
4,1.183000,11.838439
5,1.176300,13.436655
6,1.167800,12.881771
7,1.168200,13.896580
8,1.160500,14.877946
9,1.181700,13.739940
10,1.171400,14.347319


The following columns in the evaluation set don't have a corresponding argument in `RobertaForMaskedLM.forward` and have been ignored: verse_idx, name, verse, __index_level_0__. If verse_idx, name, verse, __index_level_0__ are not expected by `RobertaForMaskedLM.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 536
  Batch size = 64
Saving model checkpoint to EmbibleModels/Embibert-finetuned-tavbert-50epochs/checkpoint-500
Configuration saved in EmbibleModels/Embibert-finetuned-tavbert-50epochs/checkpoint-500/config.json
Model weights saved in EmbibleModels/Embibert-finetuned-tavbert-50epochs/checkpoint-500/pytorch_model.bin
tokenizer config file saved in EmbibleModels/Embibert-finetuned-tavbert-50epochs/checkpoint-500/tokenizer_config.json
Special tokens file saved in EmbibleModels/Embibert-finetuned-tavbert-50epochs/checkpoint-500/special_tokens_map.json
The following columns in the evaluation set don't have a corresponding argument in `Ro

TrainOutput(global_step=17350, training_loss=1.1730595463909401, metrics={'train_runtime': 2433.0261, 'train_samples_per_second': 455.194, 'train_steps_per_second': 7.131, 'total_flos': 4552990602145920.0, 'train_loss': 1.1730595463909401, 'epoch': 50.0})

In [96]:
trainer.save_model("EmbibleModels/Embibert-finetuned-tavbert-50epochs")
model.save_pretrained("EmbibleModels/Embibert-finetuned-tavbert-50epochs")

Saving model checkpoint to EmbibleModels/Embibert-finetuned-tavbert-50epochs
Configuration saved in EmbibleModels/Embibert-finetuned-tavbert-50epochs/config.json
Model weights saved in EmbibleModels/Embibert-finetuned-tavbert-50epochs/pytorch_model.bin
tokenizer config file saved in EmbibleModels/Embibert-finetuned-tavbert-50epochs/tokenizer_config.json
Special tokens file saved in EmbibleModels/Embibert-finetuned-tavbert-50epochs/special_tokens_map.json
Configuration saved in EmbibleModels/Embibert-finetuned-tavbert-50epochs/config.json
Model weights saved in EmbibleModels/Embibert-finetuned-tavbert-50epochs/pytorch_model.bin
